##### Detectron

## Region Proposals
we expect the proposals for a dataset to be provided in a pkl file that contains a serialized proposals dictionary with the following entries:

proposals.keys() - ['cfg', 'ids', 'scores', 'boxes']

where

proposals['cfg'] - config used to compute the proposals

proposals['ids][i] - id for the i-th image

proposals['scores'][i] - ndarray (N_i,) containing scores for the proposals in the i-th image

proposals['boxes'][i]  - ndarray (N_i, 4) containing coordinates (x1, y1, x2, y2) for the proposals in the i-th image


It may be instructive to download one of the proposal files from the Model Zoo and inspect it interactively. You may also find it useful to look at [_add_proposals_from_file](https://github.com/facebookresearch/Detectron/blob/master/detectron/datasets/json_dataset.py#L248) to see how proposals get loaded from file and [rpn_generator](https://github.com/facebookresearch/Detectron/blob/master/detectron/core/rpn_generator.py) to see how RPN proposals get computed and stored to a pkl file.

If you require proposals for COCO classes, you can use one of the RPN models from the Model Zoo to compute (and store in the expected format) the proposals for your dataset.

We plan to add the information about the proposals file format to the documentation in the future.
In the meantime, let us know if the above is unclear.

![Trainning process](./20180517211835738.png)

![Training](./20180517211851218.png)

==========================================================================================================================

# 1. Blobs，Workspace，Tensors

Caffe2 的 Data 是以 blobs 的形式组织的.

blob 即是内存中被命名的 data chunk(数据块).

blobs 一般包含一个 tensor(可以看做是多维数组)，在 Python 中的存在形式是 numpy arrays.

Workspace 存储所有的 blobs. 如下例，展示了将 blobs 送入 workspace (Feed) 以及从 workspace 读取 blobs [Fetch]的方法.

Workspaces 在开始使用时会自动初始化.

In [1]:
from caffe2.python import workspace, model_helper
import numpy as np
# Create random tensor of three dimensions
x = np.random.rand(4, 3, 2)
print(x)
print(x.shape)

workspace.FeedBlob("my_x", x)

x2 = workspace.FetchBlob("my_x")
print(x2)

[[[0.43033336 0.07997708]
  [0.23825377 0.54042483]
  [0.0034584  0.66832894]]

 [[0.80541561 0.53480687]
  [0.66886001 0.25899185]
  [0.34194229 0.22800033]]

 [[0.32670712 0.64598489]
  [0.1278269  0.39553063]
  [0.11457012 0.63205072]]

 [[0.98938501 0.5224455 ]
  [0.34141606 0.41883644]
  [0.45854937 0.15138438]]]
(4, 3, 2)
[[[0.43033336 0.07997708]
  [0.23825377 0.54042483]
  [0.0034584  0.66832894]]

 [[0.80541561 0.53480687]
  [0.66886001 0.25899185]
  [0.34194229 0.22800033]]

 [[0.32670712 0.64598489]
  [0.1278269  0.39553063]
  [0.11457012 0.63205072]]

 [[0.98938501 0.5224455 ]
  [0.34141606 0.41883644]
  [0.45854937 0.15138438]]]


# 2 Nets，Operators

Caffe2 基础的模型抽象是 net.

net 是由 operators 组成的 graph，每一个 operator 根据输入 input blobs 集，而输出一个或多个 output blobs.

以构建简单网络模型为例，其包括以下几种组成：

>    1 个全连接层(FC)；

>    1 个采用 softmax 的 Sigmoid 激活层；

>    1 个 CrossEntropy 层

这里不直接构建 nets，而是借助 model_helpers 来进行 nets 构建； model_helpers 是创建 nets 的 Python 类.

将待创建的网络记为 “my first net”，ModelHelper 会另外创建两个相互关联的 nets：

+ 一个是，参数初始化网络(记 init_net)
+ 一个是，真实训练网络(记 exec_net)

首先，随机生成 data 和label，并作为 blobs 输入到 workspace：


In [3]:
# Create the input data
data = np.random.rand(16, 100).astype(np.float32)

# Create labels for the data as integers [0, 9]
label = (np.random.rand(16) * 10).astype(np.int32)

workspace.FeedBlob("data", data)
workspace.FeedBlob("label", label)

print label

[3 0 0 8 9 4 8 9 3 1 3 5 1 1 6 6]


然后，采用 model_helper 来创建 init_net 和 exec_net；并接着对 model 采用 FC operator 添加全连接层. 这里 FC op 需要先生成一些随机 fills.

生成随机 fills 后，即可将 fc ops 添加到模型，并使用创建的 weights 和 bias blobs，可以根据其名字进行调用.

Caffe2 中，FC op 包括三部分： input blob，weights 和 bias. Weights 和 bias 可以使用 XavierFill 或 ConstantFill初始化，参数包括三部分： [ ] - 空数组；’fc_w/fc_b’ - 名称；shape - shape=[output, input]. 

In [4]:
# Create model using a model helper
m = model_helper.ModelHelper(name="my first net")

# Create random fills
weight = m.param_init_net.XavierFill([], 'fc_w', shape=[10, 100])
bias = m.param_init_net.ConstantFill([], 'fc_b', shape=[10, ])

fc_1 = m.net.FC(["data", "fc_w", "fc_b"], "fc1")
pred = m.net.Sigmoid(fc_1, "pred")
softmax, loss = m.net.SoftmaxWithLoss([pred, "label"], ["softmax", "loss"])

回顾下创建过程：

*        首先，创建内存里的 input data 和 label blobs；实际应用中，从相应的 database 来加载读取.

*        data 和 label blobs 的 first dim=16，即 batchsize=16.

        >基于 ModelHelper 可以进行处理许多 Caffe2 operators，更多细节参考 ModelHelper’s Operator List.

        >然后，通过定义多个 operators 来创建模型：FC, Sigmoid 和 SoftmaxWithLoss.

        >此时，只是定义了 operators 和 model.

        >ModelHelper 创建了两个 nets：
*            m.param_init_net

        > 只运行一次；初始化参数 blobs；
*            m.net 

        > 训练网络.



网络定义保存为 protobuf 结构，可以通过调用 net.Proto() 来查看网络结构.

In [5]:
print(m.net.Proto())

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label"



参数初始化化网络：

print(m.param_init_net.Proto())

# 3. Executing

定义好模型训练 operators 后，可以开始训练模型.

首先，运行一次参数初始化网络：

In [7]:
workspace.RunNetOnce(m.param_init_net)

True

然后，创建训练网络：

In [8]:
workspace.CreateNet(m.net)

True

In [21]:
# Run 100 x 10 iterations
for _ in range(100):
    data = np.random.rand(16, 100).astype(np.float32)
    label = (np.random.rand(16) * 10).astype(np.int32)

    workspace.FeedBlob("data", data)
    workspace.FeedBlob("label", label)

    workspace.RunNet(m.name, 10)   # run for 10 times
    

查看 output blobs 中存储的结果(tensor 或 numpy arrays)：

In [18]:
print(workspace.FetchBlob("softmax"))
print(workspace.FetchBlob("loss"))

[[0.0996676  0.106478   0.09701066 0.08899877 0.09087525 0.12313581
  0.08970322 0.09767637 0.07804448 0.12840985]
 [0.10409921 0.09452977 0.10418763 0.09370263 0.0992988  0.10074376
  0.10542777 0.09532675 0.07690793 0.1257758 ]
 [0.10008782 0.1073982  0.10511231 0.09442125 0.1003889  0.10805362
  0.0882118  0.10012326 0.08153807 0.11466472]
 [0.10086697 0.09680157 0.10652861 0.09293644 0.09050947 0.12851849
  0.08417885 0.09827408 0.08465172 0.11673386]
 [0.10552079 0.10716329 0.10421419 0.09145997 0.09182318 0.10571703
  0.09837991 0.09556746 0.08923232 0.11092188]
 [0.09554937 0.09340698 0.0980005  0.09848753 0.10364331 0.11400795
  0.08586583 0.09842174 0.08205641 0.13056038]
 [0.10202828 0.10348208 0.09235468 0.09227832 0.10147054 0.11293191
  0.09366265 0.10828981 0.08411993 0.10938174]
 [0.10461167 0.08678544 0.09262975 0.11165063 0.10635261 0.10574176
  0.07946953 0.10056494 0.08504988 0.12714386]
 [0.11275623 0.09755217 0.09503955 0.09639841 0.09836872 0.11127587
  0.09125371

#4. Backward pass

以上网络训练只有 forward pass，没有学习.

通过对 forward pass 中的每个 operator 创建其梯度 operators，即可 backward pass.

如果已经根据规范进行尝试，可以采用以下处理来检查结果.

*    调用 RunNetOnce 前，插入：

In [19]:
m.AddGradientOperators([loss])

{BlobReference("data"): BlobReference("data_grad"),
 BlobReference("fc_w"): BlobReference("fc_w_grad"),
 BlobReference("pred"): BlobReference("pred_grad"),
 BlobReference("loss"): BlobReference("loss_autogen_grad"),
 BlobReference("fc_b"): BlobReference("fc_b_grad"),
 BlobReference("fc1"): BlobReference("fc1_grad")}

* 检查 protobuf 输出：

In [20]:
print(m.net.Proto())

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc1"
  name: ""
  type: "FC"
}
op {
  input: "fc1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
op {
  input: "loss"
  output: "loss_autogen_grad"
  name: ""
  type: "ConstantFill"
  arg {
    name: "value"
    f: 1.0
  }
}
op {
  input: "pred"
  input: "label"
  input: "softmax"
  input: "loss_autogen_grad"
  output: "pred_grad"
  name: ""
  type: "SoftmaxWithLossGradient"
  is_gradient_op: true
}
op {
  input: "pred"
  input: "pred_grad"
  output: "fc1_grad"
  name: ""
  type: "SigmoidGradient"
  is_gradient_op: true
}
op {
  input: "data"
  input: "fc_w"
  input: "fc1_grad"
  output: "fc_w_grad"
  output: "fc_b_grad"
  output: "data_grad"
  name: ""
  type: "FCGradient"
  is_gradient_op: true
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "label